In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/stable-diffusion-keras-ft')

In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install -q datasets diffusers transformers accelerate torchmetrics[image]

In [ ]:
from textwrap import wrap
import os

import keras_cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# import tensorflow.experimental.numpy as tnp
from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.image_encoder import ImageEncoder
from keras_cv.models.stable_diffusion.noise_scheduler import NoiseScheduler
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from tensorflow import keras

In [ ]:
import numpy as np
import tensorflow as tf
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input


In [ ]:
a = pd.read_csv(os.path.join('resize_imgs', "data.csv"))

In [ ]:
!python finetune_c.py --img_height 256 --img_width 256 --batch_size 3 --num_epochs 5 --ema 0 --lr 5e-05 --augmentation --lora --lora_rank 4 --lora_alpha 8 --mp --pretrained_ckpt "b3_rank4_lr5e05/ckpt_batch_3_lora_4_lr_5e-05_epochs_10_res_256_mp_False.h5"

In [ ]:
!python finetune_c.py --img_height 256 --img_width 256 --batch_size 6 --num_epochs 10 --ema 0 --lr 5e-05 --augmentation --lora --lora_rank 4 --lora_alpha 8 --mp --pretrained_ckpt  "ckpt_batch_6_lora_4_lr_5e-05_epochs_8_res_256_mp_False.h5"

In [ ]:
!python finetune_c.py --img_height 256 --img_width 256 --lr 1e-4 --ema 0 --num_epochs 2 --lora --lora_rank 22 --lora_alpha 44 --augmentation

In [ ]:
!python finetune.py --img_height 256 --img_width 256 --batch_size 1 --num_epochs 2 --ema 0 --lora --mp --pretrained_ckpt 'ckpt_epochs_1_res_256_mp_False.h5' --lora_rank 6 --lora_alpha 4

In [ ]:
!python inference.py --prompt "an image of a 2011 Casual Fall Green Women's Shirt" --batch_size 3 --lora --lora_rank 4 --lora_alpha 8 --checkpoint 'lr5e05_ckpt_epochs_1_res_256_mp_False_lora_1.h5'

In [ ]:
from skimage import io
from torchmetrics.image.fid import FrechetInceptionDistance
from torchvision.transforms import functional as F
import torch

In [ ]:
df = pd.read_csv("resize_imgs/test_dataset.csv")

In [ ]:
def read_csv_and_return_pairs(file_path):
    df = pd.read_csv(file_path)

    if 'caption' not in df.columns or 'image_path' not in df.columns:
        raise ValueError("CSV does not contain 'k' and 'v' columns")

    pairs = df.sample(5, random_state = 41)

    return pairs.to_dict('records')

test_images = read_csv_and_return_pairs("resize_imgs/test_dataset.csv")
test_images

In [ ]:
real_ims = []
for i in range(3):
  real_im = io.imread('resize_imgs/' + test_images[i]['image_path'])
  real_ims.append(real_im)
  plt.imshow(real_im)
  plt.axis('off')
  plt.show()

In [ ]:
def read_csv_and_return_pairs(file_path):
    df = pd.read_csv(file_path)

    if 'caption' not in df.columns or 'image_path' not in df.columns:
        raise ValueError("CSV does not contain 'k' and 'v' columns")

    pairs = df.sample(5, random_state = 41)

    return pairs.to_dict('records')

test_images = read_csv_and_return_pairs("resize_imgs/test_dataset.csv")
test_images

base_images = []
for i in range(3):
  prompt = test_images[i]['caption']

  print(prompt)
  base_model = keras_cv.models.StableDiffusion(
      img_height=256, img_width=256
  )

  base_im_i = base_model.text_to_image(
      prompt=prompt,
      num_steps=50,
      batch_size=1
  )
  base_images.append(base_im_i)

In [ ]:
def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i][0])
        plt.axis("off")

plot_images(base_images)

In [ ]:

real_ims = []
for i in range(3):
  real_im = io.imread('resize_imgs/' + test_images[i]['image_path'])
  real_ims.append(real_im)



def preprocess_image(image):

    image = torch.tensor(image).unsqueeze(0)
    image = image.permute(0, 3, 1, 2) / 255.0
    return F.center_crop(image, (256, 256))


real_images = torch.cat([preprocess_image(real_im) for real_im in real_ims])
print(real_images.shape)

base_imgs = torch.cat([preprocess_image(base_i[0]) for base_i in base_images])
fid = FrechetInceptionDistance(normalize=True)
fid.update(real_images, real=True)
fid.update(base_imgs, real=False)

print(f"FID: {float(fid.compute())}")


In [ ]:
!python inference_with_fid.py --batch_size 3 --lora --lora_rank 4 --lora_alpha 8 --checkpoint 'ckpt_epochs_9_res_256_mp_False_lr_5e-05_epochs_10_cont.h5'

In [ ]:
PROMPT = "an image of a 2011 Casual Fall Black Women's Jacket"
# Load your stable diffusion model
original_model = keras_cv.models.StableDiffusion(
    img_width=256, img_height=256
)
images_original = original_model.text_to_image(PROMPT, batch_size=3)


In [ ]:
def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")


plot_images(images_original)

In [ ]:
images = model.text_to_image("a maroon strapless top", batch_size=3)


def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")


plot_images(images)

In [ ]:
PROMPT = "a green t-shirt"
BATCH_SIZE = 3
img_width = img_height = 256
original_model = keras_cv.models.StableDiffusion(
    img_width=img_width, img_height=img_height
)
images_original = original_model.text_to_image(PROMPT, batch_size=BATCH_SIZE)


In [ ]:
plot_images(images_original, "original")


In [ ]:
PROMPT = "a 2009 t-shirt"
BATCH_SIZE = 3

images_finetuned = fashion_model.text_to_image(PROMPT, batch_size=BATCH_SIZE)


plot_images(images_finetuned, "finetuned")


In [ ]:
weights_path = keras.utils.get_file(
    origin="https://huggingface.co/sayakpaul/kerascv_sd_pokemon_finetuned/resolve/main/ckpt_epochs_72_res_512_mp_True.h5"
)

img_height = img_width = 256
pokemon_model = keras_cv.models.StableDiffusion(
    img_width=img_width, img_height=img_height
)
pokemon_model.diffusion_model.load_weights(weights_path, by_name=True, skip_mismatch=True)

In [ ]:
!python inference.py --prompt "a pink strapless top" --img_height 256 --img_width 256 --batch_size 1 --checkpoint 'ckpt_epochs_20_res_256_mp_False.h5' --num_steps 20